In [1]:
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout,Flatten,Dense
from keras.models import Sequential

In [14]:
#read natis photos
# try to predict on the
%matplotlib inline

#import cv2
import matplotlib.pyplot as plt
from PIL import Image

from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
datagen = ImageDataGenerator()
#from vgg16bn import Vgg16BN
#import utils
import PIL
from keras.utils.np_utils import to_categorical
from keras.layers import Input
from keras.models import Model
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.cross_validation import train_test_split
import subprocess
import cv2
#from utils import *
from keras import backend as K
import glob
import numpy as np
import pandas as pd

In [3]:
def get_im_cv2(path, img_rows, img_cols, color_type=3):
    # Load as grayscale
    if color_type == 1:
        img = cv2.imread(path, 0)
    elif color_type == 3:
        img = cv2.imread(path)
    # Reduce size
    resized = cv2.resize(img, (img_cols, img_rows))
    return resized

def load_train_2(img_rows, img_cols, color_type=3): # - with no outer loop
    X_train = []
    y_train = []

    print('Read train images')
    #for j in unique_fish_cats:
    #print('Load folder {}'.format(j))
    #path = os.path.join('*/*.jpg')
    #files = glob.glob(path)
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl, img_rows, img_cols, color_type)

        X_train.append(img)
        j=fl.split('/')[0]
        y_train.append(j)

    return X_train, y_train

In [4]:
#change this to relevant folder
%cd /mnt/kaggle_fisherman/preprocessed_train

/mnt/kaggle_fisherman/preprocessed_train


In [5]:
files=glob.glob('*.jpg')
labels=[name.split('_')[3].split('.')[0] for name in files]
img_ids=[name.split('.')[0] for name in files]
from collections import Counter
Counter(labels)


Counter({'ALB': 1711,
         'BET': 199,
         'DOL': 117,
         'LAG': 67,
         'OTHER': 299,
         'SHARK': 175,
         'YFT': 731})

In [36]:
files[1].split('.')[0]+'.jpg'

'img_04875.jpg'

In [40]:
files

['img_01217.jpg_label_YFT.jpg',
 'img_04875.jpg_label_ALB.jpg',
 'img_03108.jpg_label_ALB.jpg',
 'img_06244.jpg_label_DOL.jpg',
 'img_02608.jpg_label_LAG.jpg',
 'img_05808.jpg_label_ALB.jpg',
 'img_03311.jpg_label_ALB.jpg',
 'img_02593.jpg_label_ALB.jpg',
 'img_05070.jpg_label_OTHER.jpg',
 'img_04220.jpg_label_ALB.jpg',
 'img_01769.jpg_label_YFT.jpg',
 'img_07570.jpg_label_OTHER.jpg',
 'img_06781.jpg_label_ALB.jpg',
 'img_01810.jpg_label_ALB.jpg',
 'img_06286.jpg_label_ALB.jpg',
 'img_05057.jpg_label_ALB.jpg',
 'img_06542.jpg_label_BET.jpg',
 'img_03661.jpg_label_ALB.jpg',
 'img_00824.jpg_label_ALB.jpg',
 'img_02732.jpg_label_SHARK.jpg',
 'img_06873.jpg_label_ALB.jpg',
 'img_06729.jpg_label_YFT.jpg',
 'img_00472.jpg_label_OTHER.jpg',
 'img_00955.jpg_label_YFT.jpg',
 'img_02742.jpg_label_BET.jpg',
 'img_00721.jpg_label_OTHER.jpg',
 'img_07167.jpg_label_ALB.jpg',
 'img_00279.jpg_label_YFT.jpg',
 'img_07891.jpg_label_YFT.jpg',
 'img_03487.jpg_label_ALB.jpg',
 'img_02097.jpg_label_BET.jpg'

In [46]:
file_names=[f.split('.')[0]+'.jpg' for f in files]

In [47]:
boat_ids

,0
0,0
1,1
2,1
3,0
4,1
5,1
6,0
7,0
8,2
9,3


In [49]:
train_file_names=[f.split('/')[-1] for f in train_files]

In [6]:
im_list=[]
for file in files:
    im_list.append(get_im_cv2(file,224,224,3))
im_array=np.array(im_list)
label_ids=[]
unique_labels=np.unique(labels)
for label in labels:
    label_ids.append(np.where(unique_labels==label)[0][0])

array([0, 1, 1, ..., 0, 6, 0])

In [7]:
im_list[0].shape,len(im_list)

((224, 224, 3), 3299)

In [48]:
## this part is taken from anokas script to identify ships
## it should not run more than once to get the boat_ids.csv file 

import pandas as pd
import numpy as np
import glob
from sklearn import cluster
from scipy.misc import imread
import cv2
import skimage.measure as sm
# import progressbar
import multiprocessing
import random
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
new_style = {'grid': False}
plt.rc('axes', **new_style)

# Function to show 4 images
def show_four(imgs, title):
    #select_imgs = [np.random.choice(imgs) for _ in range(4)]
    select_imgs = [imgs[np.random.choice(len(imgs))] for _ in range(4)]
    _, ax = plt.subplots(1, 4, sharex='col', sharey='row', figsize=(20, 3))
    plt.suptitle(title, size=20)
    for i, img in enumerate(select_imgs):
        ax[i].imshow(img)

# Function to show 8 images
def show_eight(imgs, title):
    select_imgs = [imgs[np.random.choice(len(imgs))] for _ in range(8)]
    _, ax = plt.subplots(2, 4, sharex='col', sharey='row', figsize=(20, 6))
    plt.suptitle(title, size=20)
    for i, img in enumerate(select_imgs):
        ax[i // 4, i % 4].imshow(img)

In [16]:
# Data loading
#train_files = files
from cv2 import imread
train_files = glob.glob('/mnt/kaggle_fisherman/train/*/*.jpg')
temp_train = np.array([cv2.resize(imread(img), (64, 64), cv2.INTER_LINEAR) for img in train_files])
print('Length of train {}'.format(len(temp_train)))

Length of train 3777


In [50]:
# Function for computing distance between images
def compare(args):
    img, img2 = args
    img = (img - img.mean()) / img.std()
    img2 = (img2 - img2.mean()) / img2.std()
    return np.mean(np.abs(img - img2))

# Resize the images to speed it up.
temp_train = [cv2.resize(img, (224, 224), cv2.INTER_LINEAR) for img in temp_train]

# Create the distance matrix in a multithreaded fashion
pool = multiprocessing.Pool(12)
#bar = progressbar.ProgressBar(max=len(train))
distances = np.zeros((len(temp_train), len(temp_train)))
for i, img in enumerate(temp_train): #enumerate(bar(train)):
    all_imgs = [(img, f) for f in temp_train]
    dists = pool.map(compare, all_imgs)
    distances[i, :] = dists

cls = cluster.DBSCAN(metric='precomputed', min_samples=5, eps=0.6)
boat_ids = cls.fit_predict(distances)

In [8]:
#pd.DataFrame(boat_ids).to_csv('boat_ids_correct.csv',index = False)
boat_ids=pd.read_csv('boat_ids_correct.csv')
boat_list=boat_ids['0'].values

In [10]:
from sklearn.model_selection import train_test_split

X_trn,X_val,ytrain,yval=train_test_split(im_array,np.array(label_ids),test_size=0.1)

In [34]:
#boat_ids['0'].ravel
train_files

['/mnt/kaggle_fisherman/train/NoF/img_02460.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_00494.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_00243.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_01024.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_01579.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_01135.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_00387.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_05352.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_03850.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_01775.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_06596.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_07641.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_02548.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_02524.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_05398.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_06266.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_03434.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_06366.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_04596.jpg',
 '/mnt/kaggle_fisherman/train/NoF/img_03025.jpg',


In [9]:
from sklearn.model_selection import LeaveOneGroupOut
logo = LeaveOneGroupOut()
logo.get_n_splits(im_array, label_ids, boat_list)

50

In [12]:
len(boat_list)

3777

In [10]:
i=1
stop=1
for train_index, test_index in logo.split(im_array, label_ids, boat_list):
    X_trn, X_val, ytrain,yval  = im_array[train_index], im_array[test_index], np.array(label_ids)[train_index], np.array(label_ids)[test_index]
    if i==stop:
        break
    i+=1

ValueError: Found input variables with inconsistent numbers of samples: [3299, 3299, 3777]

In [52]:
#this is a splitter from fast.ai course to separate the con layers form the dense layers

def split_at(model, layer_type):
    layers = model.layers
    layer_idx = [index for index,layer in enumerate(layers)
                 if type(layer) is layer_type][-1]
    return layers[:layer_idx+1], layers[layer_idx+1:]

In [53]:
model=VGG16()
conv_layers,fc_layers = split_at(model, Convolution2D)
for i in  range(5):
    model.layers.pop() # i did it 5 time

In [59]:
model_to_xgb.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 224, 224, 64)  1792        input_2[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 224, 224, 64)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 112, 112, 64)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [58]:
inp=Input ((224,224,3))
x = model.layers[-1].output
x = MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:])(x)
x=Flatten()(x)
model_to_xgb = Model(input = model.input,output=[x])

In [42]:
# reconstruct the model with different architecture for dense layers
p=0.5
inp=Input ((224,224,3))
x = model.layers[-1].output
x = MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:])(x)
x=BatchNormalization(axis=1)(x)
x=Dropout(p/4)(x)
x=Flatten()(x)
x=Dense(128, activation='relu')(x)
x=BatchNormalization()(x)
x=Dropout(p)(x)
x=Dense(128, activation='relu')(x)
x=BatchNormalization()(x)
x=Dropout(p/2)(x)
x=Dense(7, activation='softmax')(x)

In [43]:
model1 = Model(input = model.input,output=[x])
len(model1.layers)

29

In [44]:
#keep the convolutional layers fixed

for i,layer in enumerate(model1.layers):
    if i<18:
        model1.layers[i].trainable = False

In [45]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=180,
    rescale=2,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    horizontal_flip=True
)

In [46]:
model1.compile(Adam(lr=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model1.fit(X_trn, to_categorical(ytrain), batch_size=64 ,validation_data=(X_val,to_categorical(yval)))

In [47]:
model1.fit_generator(datagen.flow(X_trn, to_categorical(ytrain), batch_size=32)\
                     ,validation_data=(X_val,to_categorical(yval)) ,samples_per_epoch=1000,nb_epoch=10)

Epoch 1/25


/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:464: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/preprocessing/image.py:472: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn'tbeen fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


512/512 [==============================] - 31s - loss: 2.9064 - acc: 0.2285 - val_loss: 7.6193 - val_acc: 0.2860

/home/ubuntu/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


521/512 [==============================] - 31s - loss: 1.4438 - acc: 0.5739 - val_loss: 2.1559 - val_acc: 0.4899
Epoch 6/25
512/512 [==============================] - 31s - loss: 1.3065 - acc: 0.6309 - val_loss: 1.9305 - val_acc: 0.4746

KeyboardInterrupt: 

In [ ]:
#save model architecture and weights
model_json = model1.to_json()
model_path='/home/nati/data/kaggle/fish/pre-trained-model-weights/model_weights_v0.json'
with open(model_path, "w") as json_file:
    json_file.write(model_json)
    
model1.save_weights('/home/nati/data/kaggle/fish/pre-trained-model-weights/model_weights_v0.hdf5')